# Explicit Integration

It's fair to say that implicit integration techniques are the backbone of Dymos, but explicit techniques have their uses.

**When might one want to use an explicit integration technique in Dymos?**

- When physically valid results are desired from model evaluation.

    When using implicit integration techniques like those found in the Radau and GaussLobatto transcriptions, an evaluation of the model computes the collocation defects.
    It does not propagate a physically valid trajectory unless the proposed trajectory is itself physically valid.
    The use of `solve_segments=True` with the pseudospectral methods uses OpenMDAO's nonlinear solvers to converge the trajectory rather than posing them as constraints to be resolved by the optimizer, but in this case the behavior is still implicit.
    If the equations are particularly stiff, for instance, the underlying solver may have difficulty nullifying the defect residuals.  


- When data sources are noisy.

    Sometimes underlying data can be noisy.
    For instance, a thrust trace for a rocket engine being simulated is one example - it can contain hundreds of data points over the course of a few seconds.
    In these situations, it's best to smooth this data to get a model that behaves better for gradient based optimization while providing the same performance, but sometimes a user might just want Dymos to power through this model.

Explicit integration in Dymos is performed under the `ExplicitShooting` transcription.
This transcription propagates the trajectory from some initial state for some given time duration, subject to the provided parameters and controls, using explicit Runge-Kutta methods.
This document provides details of the derivation and implementation of explicit integration in Dymos.

## Fixed-Step Explicit Runge-Kutta Methods

Conider an ordinary differential equation (ODE) as posed in Dymos.
The rates of the states are a function of the current state $\bar{x}$, the current time $t$, and some set of parameters that do not change with time $\bar{\theta}$.

\begin{align}
  \dot{\bar{x}} &= f_{ODE}(\bar{x}, t, \bar{\theta})
\end{align}

For Dymos, $\bar{\theta}$ will include the standard user-provided parameters for the given ODE ($\bar{p}$), the initial time and duration of the integration, and the parameters that govern the interpolants for controls and polynomial controls. ($\hat{u}$ and $\hat{u}_p$).
That is to say, we'll derive our interpolation assuming that control interpolation happens as part of the ODE evaluation process for explicit integration.

\begin{align}
  \bar{\theta} &= \begin{bmatrix} t_0 \\ t_d \\ \bar{p} \\ \hat{u} \\ \hat{u}_p  \end{bmatrix}
\end{align}

Runge-Kutta methods consist of a _state update_ equation, a _time update_ equation, and in the case of adaptive step methods, a _stepsize update_ equation.

For the $q^{th}$ step, the state and time update equations are

\begin{align}
  \bar{x}_{q} &= \bar{x}_{q-1} + h_{q-1} \sum_{i=0}^s b_i k_{qi} & \qquad \text{state update}\\
  t_{q} &= t_{q-1} + h_{q-1} & \qquad \text{time update}    
\end{align}

The $\bar{b}$ term contains the coeffients pertaining to a given RK method.
The $\bar{k}$ term is computed for each _stage_ of a given RK method.

\begin{align}
    \bar{k}_{qi} &= f\left( t_{q-1} + c_i h_{q-1}, \bar{x}_{q-1} + h_{q-1} \sum_{j=0}^{i-1} a_{ij} k_{qj}, \bar{\theta} \right)
\end{align}

We can simplify this a bit by defining the intermediate time $T_i$ and the intermediate state $X_i$.

\begin{align}
  \bar{k}_{qi} &= f\left( \bar{X}_i, t_i, \bar{\theta} \right) \\
  \bar{X}_i &= \bar{x}_{q-1} + h_{q-1} \sum_{j=0}^{i-1} a_{ij} k_{qj} \\
  T_i &= t_{q-1} + c_i h_{q-1}
\end{align}

For the fixed-step-size case in Dymos, $h$ is determined by dividing the total time duration of the integration by the number of steps to be used $(N)$.

\begin{align}
  h(t_d) = \frac{t_d}{N}     
\end{align}

The $a_{ij}$ and $c_i$ terms are also coefficients associated with a particular Runge-Kutta scheme.
The coefficients for any given RK scheme are often layed out in a _Butcher tableau_.

\begin{align}
  \begin{array}{c|cccc}
    c_0       & a_{1,1}     & a_{1,2}    &  \dots    & a_{1,s} \\
    c_1       & a_{2,1}     & a_{2,2}    &  \dots    & a_{2,s} \\
    \vdots    & \vdots      & \vdots     &  \ddots   & \vdots \\
    c_s       & a_{s,1}     & a_{s,2}   &   \dots    & a_{s,s} \\
    \hline
    \,        & b_1        & b_2       &  \dots     & b_s
  \end{array}
\end{align}

The Butcher tableaus a few common explicit Runge-Kutta methods are below.

| Method | Tableau |
| --- | --- |
| Euler | \begin{align}
  \begin{array}{c|c}
    0         & 0 \\
    \hline
    \,        & 1      
  \end{array}
 \end{align} |
| RK4 | \begin{align}
  \begin{array}{c|cccc}
    0         & \,          & \,         &  \,        & \, \\
    1/2       & 1/2         & \,         &  \,        & \, \\
    1/2       & 0           & 1/2        &  \,        & \, \\
    1         & 0           & 0          &  1         & \, \\
    \hline
    \,        & 1/6         & 1/3        &  1/3       & 1/6
  \end{array}
\end{align} |
| Ralston | \begin{align}
  \begin{array}{c|cc}
    0         & 0   & 0 \\
    2/3       & 2/3 & 0 \\
    \hline
    \,        & 1/4 & 3/4 \\      
  \end{array}
 \end{align} |
| 3/8 | \begin{align}
  \begin{array}{c|cccc}
    0        & \,          & \,         &  \,        & \, \\
    1/3      & 1/3         & \,         &  \,        & \, \\
    2/3      & -1/3        & 1          &  \,        & \, \\
    1        & 1           & -1         &  1         & \, \\
    \hline
    \,        & 1/8         & 3/8        &  3/8       & 1/8
  \end{array}
\end{align} |



## Integration as a function

Now consider the integration process itself to be a function.
To simplify the evaluation of derivatives, we'll lump every argument to the integration into a single term ($\bar{Z}$).
The state and time update equations are now just functions of $\bar{Z}$.

\begin{align}
  \bar{Z} &= \begin{bmatrix} \bar{x}_0 \\ t \\ \bar{\theta} \end{bmatrix} = \begin{bmatrix} \bar{x}_0 \\ t \\ t_0 \\ t_d \\ \bar{p} \\ \hat{u} \\ \hat{u}_p \end{bmatrix}
\end{align}

Let $n_x$ be the total size of the state vector $\bar{x}$.  
Let $n_{\theta}$ be the total size of the ODE parameter vector $\bar{\theta}$.  
Let $n_z$ be the total size of $\bar{Z}$.  





## Derivatives of states with respect to the integration parameters

Per the state update equation we have, for a the derivative of the final state after step $q$ ($\bar{x}_q$) wrt the initial state $\bar{x}_0$:

\begin{align}
  \left[\frac{d\,\bar{x}_q}{d\,\bar{Z}}\right] &= 
    \left[\frac{\partial\,\bar{x}_q}{\partial\,\bar{Z}}\right] +
    \left[\frac{\partial\,\bar{x}_q}{\partial\,\bar{x}_{q-1}}\right] \left[\frac{d\,\bar{x}_{q-1}}{d\,\bar{Z}}\right] +
    \left[\frac{\partial\,\bar{x}_q}{\partial\,h_{q-1}}\right] \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right] +
    \sum_{i=0}^s \left[\frac{\partial\,\bar{x}_q}{\partial\,\bar{k}_{qi}}\right] \left[\frac{d\,\bar{k}_{qi}}{d\,\bar{Z}}\right]
\\
  \left[\frac{d\,\bar{x}_q}{d\,\bar{Z}}\right] &= 
    \left[0\right] +
    \left[I_n\right] \left[\frac{d\,\bar{x}_{q-1}}{d\,\bar{Z}}\right] +
    \left[ \sum_{i=0}^s b_i \bar{k}_{ni}\right] \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right] +
    h_{q-1} \sum_{i=0}^s b_i \left[\frac{d\,\bar{k}_{qi}}{d\,\bar{Z}}\right]
\end{align}

with

\begin{align}
  \left[\frac{d\,\bar{x}}{d\,\bar{Z}}\right]_0 &= \underbrace{\begin{bmatrix} & & 0 & 0 &  \dots & 0 \\
          & [I] & \vdots & \vdots & \ddots  & \vdots \\
          & & 0 & 0 & \dots & 0
  \end{bmatrix}}_{n_x \times n_z}
\\
  \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right]_0 &= \underbrace{\begin{bmatrix} 0 & \dots & 0 & \frac{1}{N} & 0 & \dots & 0 \end{bmatrix}}_{1 \times n_z}
\end{align}

That is, the left $n_x$ columns of $\left[\frac{d\,\bar{x}}{d\,\bar{Z}}\right]$ comprise the state-transition matrix, and the initial value is $[I]$.
For the initial sensitivity of the step size to the integration parameter vector, the $\frac{1}{N}$ is in the column of $\bar{Z}$ associated with the time duration of the integration, $t_d$.
For fixed-step integration this quantity does not change throughout the integration.


### Derivatives of the stage state values ($k_{qi}$) wrt the integration parameters are

\begin{align}
  \left[\frac{d\,\bar{k}_{qi}}{d\,\bar{Z}}\right] &= 
    \left[\frac{\partial\,\bar{k}_{qi}}{\partial\,\bar{Z}}\right] +
    \left[\frac{\partial\,\bar{k}_{qi}}{\partial\,\mathit{T}_i}\right] \left[\frac{d\,\mathit{T}_i}{d\,\bar{Z}}\right] +
    \left[\frac{\partial\,\bar{k}_{qi}}{\partial\,\bar{\mathit{X}}_i}\right] \left[\frac{d\,\bar{\mathit{X}}_i}{d\,\bar{Z}}\right]
\\
  \left[\frac{d\,\bar{k}_{qi}}{d\,\bar{Z}}\right] &=
    f_t\left(\mathit{T}_i, \bar{\mathit{X}}_i, \bar{\theta} \right) \left[\frac{d\,\mathit{T}_i}{d\,\bar{Z}}\right] +
    f_x\left(\mathit{T}_i, \bar{\mathit{X}}_i, \bar{\theta} \right) \left[\frac{d\,\bar{\mathit{X}}_i}{d\,\bar{Z}}\right] +
    f_{\theta}\left(\mathit{T}_i, \bar{\mathit{X}}_i, \bar{\theta} \right) \left[\frac{d\,\bar{\theta}}{d\,\bar{Z}}\right]
\end{align}

Where the $f_{\_}$ notation specifies the partial derivative (semi-total derivative in OpenMDAO terms) of the ODE function with respect to the subscripted variable.

### The derivative of the intermediate time wrt the integration parameters

\begin{align}
  \left[\frac{d\,\mathit{T}_i}{d\,\bar{Z}}\right] &=
\left[\frac{\partial\,\mathit{T}_i}{\partial\,\bar{Z}}\right] +
  \left[\frac{\partial\,\mathit{T}_{i}}{\partial\,t_{q-1}}\right] 
  \left[\frac{d\,t_{q-1}}{d\,\bar{Z}}\right] +
  \left[\frac{\partial\,\mathit{T}_{i}}{\partial\,h_{q-1}}\right] 
  \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right]
\\
  \left[\frac{d\,\mathit{T}_i}{d\,\bar{p}}\right] &= [0] + [1] \left[\frac{d\,t_{q-1}}{d\,\bar{Z}}\right] + c_i \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right]
\end{align}

with

\begin{align}
  \left[\frac{d\,t_{q-1}}{d\,\bar{Z}}\right]_0 &= 
\underbrace{\begin{bmatrix} 0 & \dots & 1 & 0 & \dots & 0 \end{bmatrix}}_{1 \times n_Z}
\end{align}

Here the $1$ is in the column assoiated with the initial time $t_0$.

### The derivatives of the intermediate states wrt the integration parameters

\begin{align}
  \left[\frac{d\,\bar{\mathit{X}}_i}{d\,\bar{Z}}\right] &=
  \left[\frac{\partial\,\bar{\mathit{X}}_i}{\partial\,\bar{Z}}\right] +
  \left[\frac{\partial\,\bar{\mathit{X}}_i}{\partial\,\bar{x}_{q-1}}\right] 
  \left[\frac{d\,\bar{x}_{q-1}}{d\,\bar{Z}}\right] +
  \left[\frac{\partial\,\bar{\mathit{X}}_i}{\partial\,h_{q-1}}\right] 
  \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right] +
  \left[\frac{\partial\,\bar{\mathit{X}}_i}{\partial\,\bar{k}_{qj}}\right] \left[\frac{d\,\bar{k}_{qj}}{d\,\bar{Z}}\right]
\\
  \left[\frac{d\,\bar{\mathit{X}}_i}{d\,\bar{Z}}\right] &= [0] +
  [I_n] \left[\frac{d\,\bar{x}_{q-1}}{d\,\bar{Z}}\right] +
  \sum_{j=0}^{i-1} a_{ij} \bar{k}_{qj} \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right] + h_{q-1} \sum_{j=0}^{i-1} a_{ij} \left[\frac{d\,\bar{k}_{qj}}{d\,\bar{Z}}\right]
\end{align}

## Derivative of time wrt the integration parameters

\begin{align}
  \left[\frac{d\,t_q}{d\,\bar{Z}}\right] &=
  \left[\frac{\partial\,t_q}{\partial\,\bar{Z}}\right] +
  \left[\frac{\partial\,t_q}{\partial\,t_{q-1}}\right] 
  \left[\frac{d\,t_{q-1}}{d\,\bar{Z}}\right] +
  \left[\frac{\partial\,t_q}{\partial\,h_{q-1}}\right] 
  \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right]
  \\
  \left[\frac{d\,t_q}{d\,\bar{Z}}\right] &=
  \left[ 0 \right] +
  \left[ 1 \right] 
  \left[\frac{d\,t_{q-1}}{d\,\bar{Z}}\right] +
  \left[ 1 \right] 
  \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right]
\end{align}

where

\begin{align} 
  \left[\frac{d\,t_{q-1}}{d\,\bar{Z}}\right]_0 &= \underbrace{\begin{bmatrix} 0 & \dots & 1 & 0 & \dots & \end{bmatrix}}_{1 \times n_z}
  \\
  \left[\frac{d\,h_{q-1}}{d\,\bar{Z}}\right]_0 &= \underbrace{\begin{bmatrix} 0 & \dots & 0& \frac{1}{N} & 0 & \dots \end{bmatrix}}_{1 \times n_z}
\end{align}

Where the $1$ is in the column of $Z$ associated with $t_0$ an the $\frac{1}{N}$ is in the column associated with $t_d$.

## Algebraic Outputs

Now suppose we have some other outputs that might be useful in an optimization context, as either constraints or objectives.

\begin{align}
  \bar{y} &= g(\bar{x}, t, \bar{\theta})
\end{align}

In dymos these are often biproducts of the calculations in the ODE and therefore computed within the ODE itself, but we'll generalize and assume it's a separate function here.

### Derivatives of the algebratic outputs

The total derivative of the algebraic outputs $(\bar{y})$ wrt the integration parameters $(\bar{Z})$ is

\begin{align}
  \left[ \frac{d\bar{y}}{d\bar{Z}} \right] &= \left[ \frac{\partial\bar{y}}{\partial\bar{Z}} \right]
    +
    \left[ \frac{\partial\bar{y}}{\partial\bar{x}} \right] \left[ \frac{d\bar{x}}{d\bar{Z}} \right]
    +
\left[ \frac{\partial\bar{y}}{\partial t} \right] \left[ \frac{d t}{d\bar{Z}} \right]
    +
\left[ \frac{\partial\bar{y}}{\partial \bar{\theta}} \right] \left[ \frac{d \bar{\theta}}{d\bar{Z}} \right] \\
  \left[ \frac{d\bar{y}}{d\bar{Z}} \right] &= 
    \left[ g_x \right] \left[ \frac{d\bar{x}}{d\bar{Z}} \right]
    +
\left[ g_t \right] \left[ \frac{d t}{d\bar{Z}} \right]
    +
\left[ g_{\theta} \right] \left[ \frac{d \bar{\theta}}{d\bar{Z}} \right]
\end{align}

## Implementation

In Dymos, explicit integration for single shooting is handled by `RKIntegrationComp`.
It propagates the states whose rates are provided by the ODE ($\dot{\bar{x}}$), in addition to accumulating the derivatives across the integration ($\left[ \frac{d\bar{x}_q}{d\bar{Z}} \right]$, $\left[ \frac{dt_q}{d\bar{Z}} \right]$).
This propagation is done by the `_propagate_vectorized_derivs` method.

Method `eval_f` is used to evaluate the ODE at a given point.
A subproblem is instantiated with its own copy of the ODE (with `num_nodes=1`)
This subproblem is then evaluated via `run_model` at the given data point, and the state rates and algebraic outputs extracted.

Evaluation of the derivatives is handled by `eval_f_derivs` and `eval_f_derivs_vectorized`.
The latter is used when evaluating the derivatives at the intermediate stages.
While the ($X_i$) at each stage depends upon the rates computed in the previous stage ($k_{qi}$), we can postpone the evaluation of the derivatives until the states and times are known at each stage.
This significantly improves performance due to the overhead of the `compute_totals` method in the underlying subproblem.

For this reason, the `RKIntegrationComp` carries two subproblem instances: `_subprob_eval` and `_subprob_derivs`, where the former is allocated with one node and the latter with a number of nodes corresponding to the number of stages in the Runge-Kutta method.

The `compute_partials` method of `RKIntegrationComp` tests to verify that the current design point was previously evaluated by `compute`.
Assuming it was, the numerical integration does not need to be done again, and it simply extracts the relevant accumulated derivatives and returns them.
This speeds up the cost of `compute_partials` by orders of magnitude, at the expense of making the `compute` method roughly twice as expensive, due to the added cost of accumulating the derivatives during every propagation, whether they will be wanted or not.

## The ODE Subproblem

The ODE subproblem takes the current states and times, as well as the parameters, control parameters, and polynomial control parameters.
The XDSM below shows the flow of data in the ODE subproblem.
The ODE subproblem includes interpolation of the dynamic controls by a Vandermonde interpolation technique.
To avoid numerical issues within the Vandermonde interpolation approach, interpolation is done with respect to the nondimensional time across each segment ($\tau_s$).
The ODE subproblem therefore contains four subsystems:
1. Tau calculation based on the current time.
2. Vandermonde interpolation of the dynamic controls.
3. The ODE subsystem.
4. A collection component for state rates that obtains their values regardless of the source.


